In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tornado.websocket import WebSocketProtocol13
%matplotlib inline

In [2]:
#read all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)


32033

In [4]:
# build the vocab of characters and mapping to/from integers
chars = sorted(list(set(''.join(words))))
stoi= {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)
vocab_size = len(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [5]:
block_size = 3
def build_dataset(words):
    # build the dataset
    
    X,Y = [],[]
    for w in words:

        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context),'--------->',itos[ix])
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X,Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
Xtr,Ytr = build_dataset(words[:n1])
Xdev,Ydev = build_dataset(words[n1:n2])
Xte,Yte = build_dataset(words[n2:])


In [6]:
print(Xtr.shape,Ytr.shape)
print(Xdev.shape,Ydev.shape)
print(Xte.shape,Yte.shape)

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [7]:
n_embd = 10 # dimension of character embedding vectors

n_hidden = 200

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size,n_embd),generator=g)
W1 = torch.randn((n_embd*block_size,n_hidden),generator=g) * 0.2
b1 = torch.randn(n_hidden,generator=g) *0.01
W2 = torch.randn((n_hidden,vocab_size),generator=g) * 0.01
b2 = torch.randn(vocab_size,generator=g) * 0

bngain = torch.ones((1,n_hidden))
bnbias = torch.zeros((1,n_hidden))

parameters = [C,W1,b1,W2,b2,bngain,bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

12297


In [9]:
max_steps = 200000
batch_size = 32
lossi= []
for i in range(max_steps):
    # mini-batch construct
    ix = torch.randint(0,Xtr.shape[0],(batch_size,),generator=g)
    Xb,Yb = Xtr[ix],Ytr[ix]
    
    # forward pass
    emb = C[Xb] # characters embeddings
    embcat = emb.view(emb.shape[0],-1)
    hpreact = embcat @ W1 + b1 # hidden layer before activation
    
    hpreact = bngain*(hpreact-hpreact.mean(0,keepdim=True)) / hpreact.std(0,keepdim=True) + bnbias
    h = torch.tanh(hpreact)
    logits = h @ W2 + b2 #output layer
    loss = F.cross_entropy(logits,Yb) # loss function
    
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    
    # update
    lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
    for p in parameters:
        p.data += -lr * p.grad
    
    # track stats
    
    if i % 10000 == 0: # print
        print(f'{i:7d} / {max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item)
    
    
    

      0 /  200000: 3.3147
  10000 /  200000: 2.1341
  20000 /  200000: 2.3173
  30000 /  200000: 2.4115
  40000 /  200000: 2.0124
  50000 /  200000: 2.3003
  60000 /  200000: 2.4708
  70000 /  200000: 2.1208
  80000 /  200000: 2.3562
  90000 /  200000: 2.1095
 100000 /  200000: 1.9329
 110000 /  200000: 2.3919
 120000 /  200000: 1.9795
 130000 /  200000: 2.4764
 140000 /  200000: 2.3703
 150000 /  200000: 2.3098
 160000 /  200000: 2.0003
 170000 /  200000: 1.8345
 180000 /  200000: 2.0355
 190000 /  200000: 1.8853


In [10]:
hpreact.shape

torch.Size([32, 200])

In [ ]:
##### Kendim denedim
a = logits[0]
b = a *0.01
print(a)
print(b)
counts_a= a.exp()
counts_b= b.exp()
probs_a = counts_a / counts_a.sum()
probs_b = counts_b / counts_b.sum()
print(probs_a)
print(-probs_a[12].log())
print(probs_b)
print(-probs_b[12].log())

In [ ]:
@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
    x,y = {
        'train':(Xtr,Ytr),
        'val':(Xdev,Ydev),
        'test':(Xte,Yte)
    }[split]
    emb = C[x]
    embcat = emb.view(emb.shape[0],-1)
    hpreact = embcat @ W1 + b1
    hpreact =bngain*(hpreact-hpreact.mean(0,keepdim=True)) / hpreact.std(0,keepdim=True) + bnbias
    h = torch.tanh(embcat @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits,y)
    print(loss.item())

split_loss('train')
split_loss('val')
    

In [ ]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
        emb = C[torch.tensor([context])] # (1,block_size,d)
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break

    print(''.join(itos[i] for i in out))